# 01 - 批次生成對話資料

<div align="left" style="line-height: 1;">
  <a href="https://discord.gg/Cx737yw4ed" target="_blank" style="margin: 2px;">
    <img alt="Discord" src="https://img.shields.io/badge/Discord-Twinkle%20AI-7289da?logo=discord&logoColor=white&color=7289da" style="display: inline-block; vertical-align: middle;"/>
  </a>
  <a href="https://huggingface.co/twinkle-ai" target="_blank" style="margin: 2px;">
    <img alt="Hugging Face" src="https://img.shields.io/badge/%F0%9F%A4%97%20Hugging%20Face-Twinkle%20AI-ffc107?color=ffc107&logoColor=white" style="display: inline-block; vertical-align: middle;"/>
  </a>
</div>

在這個 Notebook 中，你將學會：
- 規劃「主題池 / 風格 / 隨機種子 / 批量數量」
- 設計 prompt 模板，控制輸出品質與格式
- 使用 OpenAI SDK（相容端點）批量呼叫模型
- 將結果保存為 `JSONL`（後續會做品質檢查與格式化）

> 提醒：這支 Notebook 預期你已完成 **00_setup_and_api_call**。

## 1. 參數設定與端點初始化

- `API_KEY`：請向 Twinkle AI 社群索取。  
- `BASE_URL`：使用提供的相容端點。  
- `MODEL`：本課使用 `gemma-3-12b-it`。  
- 其他生成參數：`temperature`、`max_tokens`、`seed`、`n_per_topic` 等。

In [3]:
from openai import OpenAI
import os, json, random
from pathlib import Path
from typing import List, Dict

# === 端點與金鑰（依課程提供填入） ===
API_KEY = "sk-eT_04m428oAPUD5kUmIhVA"  # 註解：這裡要問 Twinkle AI 社群
BASE_URL = "https://litellm-ekkks8gsocw.dgx-coolify.apmic.ai"
MODEL = "gemma-3-12b-it"

client = OpenAI(api_key=API_KEY, base_url=f"{BASE_URL}/v1")

# === 生成參數 ===
SEED = 42                      # 控制可重現性
TEMPERATURE = 0.7              # 多樣性
MAX_TOKENS = 800               # 允許較長的多輪對話
TOPICS = ["金融客服", "醫療諮詢", "法律諮詢", "電商售後", "教育輔導"]
STYLE = "正式且友善，使用繁體中文，語句清楚、避免冗長。"
N_PER_TOPIC = 5                # 每個主題生成幾筆
OUTPUT_DIR = Path("outputs")
OUTPUT_PATH = OUTPUT_DIR / "raw.jsonl"

random.seed(SEED)
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
print("✅ 初始化完成，將輸出到：", OUTPUT_PATH)

✅ 初始化完成，將輸出到： outputs/raw.jsonl


## 2. 設計 Prompt 模板

我們使用 `system` + `user`：
- **system**：定義助理角色與語氣（專業、友善、繁中），並要求輸出為「多輪對話」與「清晰結構」。
- **user**：傳入「主題」「風格」與「具體要求」。例如：
  - 至少 3 輪（user/assistant 交替）
  - 禁止個資（姓名、電話、住址、身分證等）
  - 建議包含「釐清需求 → 提供步驟/要點 → 總結」

> 小提醒：清楚的模板能顯著提升資料可用性與後續清理效率。

In [4]:
SYSTEM_PROMPT = (
    "你是專業的對話助理，使用繁體中文回答。請保持語氣正式且友善，"
    "輸出內容要有邏輯結構，避免冗長與重複。"
)

USER_TEMPLATE = (
    "請依據主題生成一段『真實世界』的多輪對話，至少 3 輪，角色為使用者 (user) 與助理 (assistant) 交替。\n"
    "主題：{topic}\n"
    "風格：{style}\n"
    "要求：\n"
    "1) 禁止包含任何個人資料或敏感資訊（例如姓名、電話、住址、身分證號）。\n"
    "2) 對話需包含：釐清需求 → 提供步驟或要點 → 簡短總結。\n"
    "3) 內容須可作為微調資料，避免使用 placeholder（如 XXX）。\n"
    "4) 只輸出對話本身，不要額外說明。"
)

## 3. 呼叫函式與重試機制

為了提升穩定性，我們包一層簡單的重試（網路/暫時性錯誤時退避重試）。  
回傳內容以「對話文字」為主，後續我們會存為 `messages` 結構。

In [5]:
import time

def chat_completion(messages: List[Dict], temperature: float = TEMPERATURE, max_tokens: int = MAX_TOKENS, retries: int = 3):
    for attempt in range(1, retries + 1):
        try:
            resp = client.chat.completions.create(
                model=MODEL,
                messages=messages,
                temperature=temperature,
                max_tokens=max_tokens,
            )
            return resp.choices[0].message.content
        except Exception as e:
            if attempt == retries:
                raise
            sleep_s = 1.5 * attempt
            print(f"⚠️ 呼叫失敗（第 {attempt} 次）：{e}，{sleep_s:.1f}s 後重試…")
            time.sleep(sleep_s)

## 4. 單筆資料生成器

把主題與風格帶入模板，組成 `messages` 後呼叫 API。  
回傳結構採最小可用格式：
```json
{
  "id": "<隨機ID>",
  "topic": "<主題>",
  "style": "<風格>",
  "messages": [
    {"role": "system", "content": "..."},
    {"role": "user", "content": "..."},
    {"role": "assistant", "content": "...(模型輸出對話文本)..."}
  ]
}

In [6]:
import uuid

def generate_example(topic: str, style: str = STYLE) -> Dict:
    user_prompt = USER_TEMPLATE.format(topic=topic, style=style)
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": user_prompt},
    ]
    content = chat_completion(messages)
    example = {
        "id": str(uuid.uuid4()),
        "topic": topic,
        "style": style,
        "messages": [
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": user_prompt},
            {"role": "assistant", "content": content},
        ],
    }
    return example

## 5. 批次生成與進度顯示

我們會針對每個主題生成 `N_PER_TOPIC` 筆資料，並即時寫入 `raw.jsonl`：  
- 一筆一行，方便後續處理（品質檢查 / 轉檔）。  
- 若中途中斷，可藉由檔案內容判斷已完成數量。

In [7]:
from tqdm.auto import tqdm

# 若已存在舊檔，建議先備份或刪除
if OUTPUT_PATH.exists():
    print(f"⚠️ {OUTPUT_PATH} 已存在，將覆寫。")

count = 0
with OUTPUT_PATH.open("w", encoding="utf-8") as f:
    for topic in tqdm(TOPICS, desc="Topics"):
        for _ in tqdm(range(N_PER_TOPIC), leave=False, desc=f"Generating for {topic}"):
            ex = generate_example(topic)
            f.write(json.dumps(ex, ensure_ascii=False) + "\n")
            count += 1

print(f"✅ 生成完成，共 {count} 筆，已寫入 {OUTPUT_PATH}")

/opt/conda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Topics: 100%|██████████████████████████████████████████| 5/5 [00:23<00:00,  4.69s/it]

✅ 生成完成，共 25 筆，已寫入 outputs/raw.jsonl


## 6. 快速抽樣檢視

讀取部分樣本，確認結構與內容是否符合預期（詳細的敏感詞/長度/結構檢查會在下一支 Notebook 進行）。

In [9]:
samples = []
with OUTPUT_PATH.open("r", encoding="utf-8") as f:
    for i, line in enumerate(f):
        if i >= 3:  # 只看前三筆
            break
        samples.append(json.loads(line))

for i, s in enumerate(samples, 1):
    print(f"\n--- Sample {i} / topic={s['topic']} ---")
    # 只截斷顯示前 500 字，避免輸出過長
    text = s["messages"][-1]["content"]
    print(text[:500] + ("..." if len(text) > 500 else ""))


--- Sample 1 / topic=金融客服 ---
**第一輪**

**user:** 您好，我想詢問關於我的定期定額投資，最近的扣款出現了失敗的狀況，想知道如何處理。

**assistant:** 您好，感謝您的聯繫。關於定期定額扣款失敗的問題，我們很樂意為您提供協助。為了更準確地了解情況，請問您的扣款失敗是針對哪個投資組合呢？以及您是否有收到任何來自銀行的扣款失敗通知，例如失敗原因？

**第二輪**

**user:** 是針對「全球永續成長」的投資組合，我收到銀行通知，顯示是因帳戶餘額不足導致扣款失敗。

**assistant:** 好的，感謝您提供詳細資訊。帳戶餘額不足是常見的扣款失敗原因。針對此情況，我們建議您採取以下步驟：

1.  **確認帳戶餘額：** 請您再次確認您的銀行帳戶餘額是否已足夠支付當期的定期定額金額。
2.  **重新安排扣款：** 您可以透過銀行App或網銀，重新安排當期扣款。通常系統會保留扣款紀錄，您可以選擇在餘額充足後重新嘗試。
3.  **調整定額金額或週期：** 如果您經常遇到餘額不足的情況，建議您考慮調整定期定額的金額或扣款週期，以符合您的財務狀況。
4.  **聯繫銀行：** 若重新安...

--- Sample 2 / topic=金融客服 ---
**第一輪**

**user:** 您好，我想詢問關於我的定期定額投資，最近的扣款出現了失敗的狀況，想知道如何處理。

**assistant:** 您好，感謝您的聯繫。關於定期定額扣款失敗的問題，我們很樂意為您提供協助。為了更準確地了解情況，請問您的扣款失敗是針對哪個投資組合呢？以及您是否有收到任何來自銀行的扣款失敗通知，例如失敗原因？

**第二輪**

**user:** 是針對「全球永續成長」的投資組合，我收到銀行通知，顯示是因帳戶餘額不足導致扣款失敗。

**assistant:** 好的，感謝您提供詳細資訊。帳戶餘額不足是常見的扣款失敗原因。針對此情況，我們建議您採取以下步驟：

1.  **確認帳戶餘額：** 請您再次確認您的銀行帳戶餘額是否已足夠支付當期的定期定額金額。
2.  **重新安排扣款：** 您可以透過銀行App或網銀，重新安排當期扣款。通常系統會保留扣款紀錄，您可以選擇在餘額充足後重新嘗試。
3.  **調整定額金額或週期：*

## 7. 小挑戰（練習）

1. 調整 `STYLE` 與 `TEMPERATURE`，觀察輸出差異。  
2. 增加/替換 `TOPICS`，例如加入「公共服務」「製造業維運」。  
3. 嘗試提高 `N_PER_TOPIC`，建立更大的 raw 資料集。  
4. 在 `USER_TEMPLATE` 中加入更多結構化約束（例如要求分段或使用標題）。  